In [1]:
pip install nsetools

Note: you may need to restart the kernel to use updated packages.


In [2]:
from nsetools import Nse
nse = Nse()

In [3]:
q = nse.get_quote('INFY')
q['isinCode']

'INE009A01021'

In [4]:
import pandas as pd
get_stock_ticker = pd.read_csv('https://raw.githubusercontent.com/smeet-kothari/SM-Portfolio-Manager/master/EQUITY_L.csv')
get_stock_ticker.head()

,SYMBOL,NAME OF COMPANY,SERIES,DATE OF LISTING,PAID UP VALUE,MARKET LOT,ISIN NUMBER,FACE VALUE
0,20MICRONS,20 Microns Limited,EQ,06-OCT-2008,5,1,INE144J01027,5
1,21STCENMGM,21st Century Management Services Limited,EQ,03-MAY-1995,10,1,INE253B01015,10
2,3IINFOTECH,3i Infotech Limited,EQ,22-APR-2005,10,1,INE748C01020,10
3,3MINDIA,3M India Limited,EQ,13-AUG-2004,10,1,INE470A01017,10
4,3PLAND,3P Land Holdings Limited,EQ,19-JUL-1995,2,1,INE105C01023,2


In [ ]:
def predict_infosys():
        # -*- coding: utf-8 -*-
    """LSTM.ipynb

    Automatically generated by Colaboratory.

    Original file is located at
        https://colab.research.google.com/drive/1H0AM71_a1ZLfW5gXg9Mhf9jP8uh8P0La
    """

    #Long Short Term Memory

    import math
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import pandas_datareader as web
    plt.style.use('fivethirtyeight')

    #pip install tensorflow

    from sklearn.preprocessing import MinMaxScaler
    from keras.models import Sequential 
    from keras.layers import Dense, LSTM

    #https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92

    df = pd.read_csv('https://raw.githubusercontent.com/smeet-kothari/SM-Portfolio-Manager/master/11-04-2018-TO-09-04-2020INFYEQN.csv')

    df.head()

    df = df.iloc[ :, 8:9 ]

    df.head()

    dataset = df.values

    dataset[ :5 ]

    #Scaling data
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(dataset)

    # Training Dataset
    training_length = math.ceil(0.8 *len(scaled_data))
    train_data = scaled_data[ :training_length]
    #split data into x_train and y_train
    x_train = []
    y_train = []

    for i in range(60,len(train_data)):
        x_train.append(train_data[i-60:i,0])
        y_train.append(train_data[i,0])
        if i<= 60:
            print(x_train)
            print(y_train)
            print()

    #x_train and y_train to numpy arrays
    x_train = np.array(x_train)
    y_train = np.array(y_train)

    x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1) )
    x_train.shape

    #LSTM

    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1],x_train.shape[2]) ) )
    model.add(LSTM(50, return_sequences=False) )
    model.add(Dense(25))
    model.add(Dense(1))

    #Compile the Model

    model.compile(optimizer='adam', loss='mean_squared_error')

    #Train

    model.fit(x_train,y_train,batch_size=1,epochs=1)

    # Testing Data Set

    test_data = scaled_data[training_length-60: , : ]

    #Data Sets
    x_test = []
    y_test = dataset[training_length: , : ]

    for i in range(60,len(test_data)):
        x_test.append(test_data[i-60:i, 0])

    x_test = np.array(x_test)

    #Reshape the dataset

    x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1) )

    #Get the model Predicted Value

    predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(predictions)

    #Root Mean Sqaure error

    rmse = np.sqrt( np.mean( predictions-y_test)**2 )
    rmse

    train = df[ :training_length]
    valid = df[training_length: ]

    valid.head()

    """valid.loc['Predictions'] =predictions
    #Visulization

    plt.title('Model')
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.plot(train.loc['Close Price'])
    plt.show(valid.loc[['Close Price' , 'Predictions']])
    plt.legend(['Train','Val','Predictions'], loc='upper_right' )"""

    new_df = pd.read_csv('https://raw.githubusercontent.com/smeet-kothari/SM-Portfolio-Manager/master/11-04-2018-TO-09-04-2020INFYEQN.csv')

    new_df = new_df.iloc[ :, 8:9 ]

    new_df.head()

    last_60_days = new_df[-60:].values

    last_60_days_scaled = scaler.transform(last_60_days)

    X_test = []
    X_test.append(last_60_days_scaled)

    X_test = np.array(x_test)

    X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1) )

    pred_price = model.predict(X_test)
    pred_price = scaler.inverse_transform(pred_price)

    return pred_price[-1]

In [6]:
def build_bse_infosys_df():
    raw_data=pd.read_html("https://www.moneycontrol.com/stocks/marketstats/industry-classification/nse/it-services-consulting.html")

    df_bse_infosys = raw_data[0]

    df_bse_infosys[['Company','others']]= df_bse_infosys['Company Name'].str.split(r"\bAdd\b",expand=True)
    df_bse_infosys['Company Name']=df_bse_infosys['Company']

    df_bse_infosys= df_bse_infosys.iloc[ 133, :7:]
                          
    df_bse_infosys['Category'] = "IT-Services-Consulting"

    return df_bse_infosys
    
df = build_bse_infosys_df()

df['ISIN NUMBER'] = 'INE009A01021'

#df['Close Prediction'] = predict_infosys()
df['Close Prediction'] = 649.8704

df = pd.DataFrame(df)
                          
print(df)

                                     133
Company Name                   Infosys  
Open                              629.00
High                               651.7
Low                               626.05
Last Price                         637.4
Prev Price                        636.25
Change                              1.15
Category          IT-Services-Consulting
ISIN NUMBER                 INE009A01021
Close Prediction                  649.87
